In [1]:
from distribution_fit_class import DistributionFit
from portfolio_optimization_class import PortfolioOptimization, optimize_windows
import pandas as pd
import numpy as np

In [2]:
returns_df = pd.read_csv(r'data\random_data\n_stocks_per_sector.csv', index_col=0, parse_dates=True)
file_path = r'data\random_data\n_stocks_per_sector.csv'

copulas = ["clayton_random", "gaussian", "t_student"]
distributions = ["gauss_dist", "t_dist"]
window_size = 32
taus = [0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2]
maximum_weight = 0.3

In [31]:
aggregated_results = []

for tau in taus:

    portfolios = optimize_windows(number_of_quarters=window_size, tau=tau, maximum_weight=maximum_weight, data_path=file_path)
    model_aggregation = []
    for window in range(3, 63):  # 60 windows as specified
        if window % 5 == 0:
            print(f'Processing window: {window}, tau: {tau}')

        # Select returns window for fitting distributions
        returns_window = returns_df.iloc[window: 32 + window_size]
        distribution_fit = DistributionFit()
        distribution_fit.set_df(returns_window)
        distribution_fit.fit_multivariate_distributions(4)  # Fit the multivariate distributions

        # Generate samples from fitted distributions
        distribution_fit.generate_multivariate_normal_samples(10000)
        distribution_fit.generate_multivariate_t_samples(10000)

        samples_normal = distribution_fit.get_generated_multivariated_normal_samples()
        samples_t = distribution_fit.get_generated_multivariated_t_samples()


        distributions_samples = [
            (samples_normal, "gauss_dist"),
            (samples_t, "t_dist")
        ]

        for simulated_returns, dist_name in distributions_samples:
            weights = portfolios.iloc[window].filter(regex="^w\d+$").values
            expectile = -portfolios.iloc[window]["EVAR"]
            weighted_returns = simulated_returns.values * weights

            validation_values = (1 - tau) * np.minimum(weighted_returns - expectile, 0) - tau * np.maximum(weighted_returns - expectile, 0)
            scoring_values = (1 - tau) * np.minimum((weighted_returns - expectile)**2, 0) + tau * np.maximum((weighted_returns - expectile)**2, 0)

            expected_validation_values = np.mean(validation_values, axis=1)
            expected_scoring_values = np.mean(scoring_values, axis=1)

            validation_df = pd.DataFrame({
                'Portfolio Return': np.sum(weighted_returns, axis=1),
                'Expected Validation Value': expected_validation_values,
                'Expected Scoring Value': expected_scoring_values
            })
            # print(validation_df)

            statistics = {
                        'Window': window,
                        'Distribution': dist_name,
                        'mean_validation': validation_df['Expected Validation Value'].mean(),
                        'std_validation': validation_df['Expected Validation Value'].std(),
                        'p1_validation': np.percentile(validation_df['Expected Validation Value'], 1),
                        'p5_validation': np.percentile(validation_df['Expected Validation Value'], 5),
                        'p95_validation': np.percentile(validation_df['Expected Validation Value'], 95),
                        'p99_validation': np.percentile(validation_df['Expected Validation Value'], 99),

                        'mean_scoring': validation_df['Expected Scoring Value'].mean(),
                        'std_scoring': validation_df['Expected Scoring Value'].std(),
                        'p1_scoring': np.percentile(validation_df['Expected Scoring Value'], 1),
                        'p5_scoring': np.percentile(validation_df['Expected Scoring Value'], 5),
                        'p95_scoring': np.percentile(validation_df['Expected Scoring Value'], 95),
                        'p99_scoring': np.percentile(validation_df['Expected Scoring Value'], 99),
                    }
            model_aggregation.append(statistics)
    tau_df = pd.DataFrame(model_aggregation)
    aggregated_results.append(tau_df)




Processing window: 5, tau: 0.0005
Processing window: 10, tau: 0.0005
Processing window: 15, tau: 0.0005
Processing window: 20, tau: 0.0005
Processing window: 25, tau: 0.0005
Processing window: 30, tau: 0.0005
Processing window: 35, tau: 0.0005
Processing window: 40, tau: 0.0005
Processing window: 45, tau: 0.0005
Processing window: 50, tau: 0.0005
Processing window: 55, tau: 0.0005
Processing window: 60, tau: 0.0005
Processing window: 5, tau: 0.001
Processing window: 10, tau: 0.001
Processing window: 15, tau: 0.001
Processing window: 20, tau: 0.001
Processing window: 25, tau: 0.001
Processing window: 30, tau: 0.001
Processing window: 35, tau: 0.001
Processing window: 40, tau: 0.001
Processing window: 45, tau: 0.001
Processing window: 50, tau: 0.001
Processing window: 55, tau: 0.001
Processing window: 60, tau: 0.001
Processing window: 5, tau: 0.002
Processing window: 10, tau: 0.002
Processing window: 15, tau: 0.002
Processing window: 20, tau: 0.002
Processing window: 25, tau: 0.002
Proce

In [20]:
aggregated_results[0]

,Window,Distribution,mean_validation,std_validation,p1_validation,p5_validation,p95_validation,p99_validation,mean_scoring,std_scoring,p1_scoring,p5_scoring,p95_scoring,p99_scoring
0,3,gauss_dist,-0.000124,3.199793e-04,-0.001514,-0.000841,-0.000012,-0.000011,4.249334e-07,1.171867e-07,2.603176e-07,2.862240e-07,6.420528e-07,7.801531e-07
1,3,t_dist,-0.000449,1.256430e-03,-0.005990,-0.002585,-0.000012,-0.000012,4.840195e-07,2.477317e-07,2.568543e-07,2.901831e-07,8.409205e-07,1.556201e-06
2,4,gauss_dist,-0.000117,3.269801e-04,-0.001725,-0.000764,-0.000012,-0.000011,4.249545e-07,1.202640e-07,2.618699e-07,2.783540e-07,6.683395e-07,8.190250e-07
3,4,t_dist,-0.000367,1.414398e-03,-0.004620,-0.002187,-0.000012,-0.000011,4.771960e-07,2.482019e-07,2.579161e-07,2.807850e-07,8.418819e-07,1.305103e-06
4,5,gauss_dist,-0.000190,6.707643e-04,-0.003431,-0.001191,-0.000028,-0.000027,2.070484e-06,4.281897e-07,1.461509e-06,1.550373e-06,2.901566e-06,3.332267e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,60,t_dist,-0.000148,7.416367e-04,-0.003356,-0.000056,-0.000052,-0.000051,5.825636e-06,3.364373e-07,5.344829e-06,5.493253e-06,6.330540e-06,6.861159e-06
116,61,gauss_dist,-0.000062,1.083268e-04,-0.000091,-0.000054,-0.000052,-0.000052,5.736253e-06,2.003907e-07,5.445591e-06,5.492745e-06,6.096614e-06,6.443747e-06
117,61,t_dist,-0.000280,1.925880e-03,-0.004793,-0.000624,-0.000052,-0.000051,5.889035e-06,1.565789e-06,5.424013e-06,5.488949e-06,6.445578e-06,7.577893e-06
118,62,gauss_dist,-0.000053,7.231911e-07,-0.000054,-0.000054,-0.000052,-0.000051,5.615518e-06,1.448204e-07,5.373725e-06,5.415661e-06,5.874762e-06,6.033748e-06


In [32]:
def evaluate_gauss_t_model_performance(aggregated_results, taus):
    tau_evaluation_results = {}

    for tau_idx, tau_value in enumerate(taus):
        evaluation_results = []
        tau_df = aggregated_results[tau_idx]

        for dist in distributions:
            filtered_df = tau_df[tau_df["Distribution"] == dist]

            evaluation_results.append({
                'Distribution': dist,
                'Mean_Val': filtered_df['mean_validation'].mean(),
                'Std_Val': filtered_df['std_validation'].mean(),
                'P5_Val': filtered_df['p5_validation'].mean(),
                'P95_Val': filtered_df['p95_validation'].mean(),
                'Mean_Sco': filtered_df['mean_scoring'].mean(),
                'Std_Sco': filtered_df['std_scoring'].mean(),
                'P5_Sco': filtered_df['p5_scoring'].mean(),
                'P95_Sco': filtered_df['p95_scoring'].mean(),
            })
        evaluation_df = pd.DataFrame(evaluation_results)
        evaluation_df = evaluation_df.sort_values(by='Mean_Val')
        tau_evaluation_results[tau_value] = evaluation_df
    return tau_evaluation_results


In [33]:
tau_evaluation_results = evaluate_gauss_t_model_performance(aggregated_results, taus)

In [29]:
tau_evaluation_results[0.0005]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.000298,0.001205,-0.001514,-0.000028,0.000002,5.664367e-07,0.000002,0.000003
0,gauss_dist,-0.000096,0.000260,-0.000453,-0.000028,0.000002,1.999011e-07,0.000002,0.000003


In [42]:
tau_evaluation_results[0.001]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.000327,0.001147,-0.001584,-0.000055,0.000004,1.176410e-06,0.000004,0.000006
0,gauss_dist,-0.000125,0.000263,-0.000462,-0.000055,0.000004,3.979379e-07,0.000004,0.000005


In [41]:
tau_evaluation_results[0.002]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.000390,0.001149,-0.001667,-0.000107,0.000009,1.957530e-06,0.000007,0.000011
0,gauss_dist,-0.000187,0.000274,-0.000555,-0.000107,0.000008,7.868738e-07,0.000007,0.000010


In [40]:
tau_evaluation_results[0.005]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.000563,0.001191,-0.001908,-0.000250,0.000019,0.000005,0.000016,0.000024
0,gauss_dist,-0.000351,0.000285,-0.000775,-0.000249,0.000018,0.000002,0.000016,0.000022


In [39]:
tau_evaluation_results[0.01]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.000815,0.001227,-0.002253,-0.000449,0.000031,0.000010,0.000026,0.000041
0,gauss_dist,-0.000588,0.000304,-0.001080,-0.000448,0.000030,0.000003,0.000026,0.000036


In [38]:
tau_evaluation_results[0.02]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.001307,0.001298,-0.002992,-0.000820,0.000044,0.000016,0.000035,0.000062
0,gauss_dist,-0.001037,0.000353,-0.001676,-0.000816,0.000042,0.000006,0.000035,0.000052


In [37]:
tau_evaluation_results[0.05]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.002447,0.001540,-0.004878,-0.001590,0.000044,0.000038,0.000027,0.000078
0,gauss_dist,-0.002085,0.000568,-0.003225,-0.001584,0.000038,0.000010,0.000027,0.000056


In [36]:
tau_evaluation_results[0.1]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.004782,0.002070,-0.008319,-0.003187,0.000045,0.000099,0.000011,0.000125
0,gauss_dist,-0.004297,0.001016,-0.006310,-0.003183,0.000029,0.000019,0.000010,0.000067


In [35]:
tau_evaluation_results[0.15]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.011528,0.002449,-0.015829,-0.009189,0.000088,0.000157,0.000032,0.000225
0,gauss_dist,-0.011003,0.001434,-0.013730,-0.009175,0.000060,0.000030,0.000032,0.000121


In [34]:
tau_evaluation_results[0.2]

,Distribution,Mean_Val,Std_Val,P5_Val,P95_Val,Mean_Sco,Std_Sco,P5_Sco,P95_Sco
1,t_dist,-0.019172,0.002807,-0.024084,-0.016268,0.000204,0.000291,0.000112,0.000425
0,gauss_dist,-0.018612,0.001784,-0.021958,-0.016231,0.000159,0.000048,0.000111,0.000252
